# Copiar documentos de Google Forms y posterior cambiar nombre

El siguiente script con Python, permite copiar los archivos cargados en un formulario de Google Forms a carpetas por participantes u organización y en el mismo proceso, cambia el nombre del documento al deseado por cada proyecto

A continuación se describen los pasos para ejecutar el script

# 1. Conexión Google Drive

Por medio de la libreria otorgada por Google, se realiza la connexión de Google Colaboratory y Google Drive y así acceder a los formularios creados

In [1]:
# para montar datos desde google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Principales librerias

Se importan las siguientes librerias

*   Se importan las librerias para analisis de datos (Pnadas y Numpy)
*   Autorización Google Cola con Google Sheets por medio de Google SDK

In [2]:
import os
import pandas as pd
import numpy as np
import requests
import re

#Google Sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [ ]:
#!ls -l /content/drive/MyDrive/

# 3. Cargue información formulario - Google Sheets

Una vez se autorice con Google SDK, se debe ingresar el nombre de la hoja de Google Sheet que tenga la infromación del formulario, esto servira para lo siguiente:

*   Identificar la información del participante o la organización que se quere mover 
*   Identificar los id de las carpetas y los archivos con los que se va atrabajar



In [3]:
## Coloque el nombre nameSheets Ejemplo: 'Prueba_cambio_nombre (respuestas)'
## Este automaticamente detectara la información de los formularios que se deseen trabajar

### Colocar el nombre del sheet
nameSheets = input()


Prueba (respuestas)


In [4]:
## Con el siguiente proceso, realizara una identificación de la información que tiene el Google Forms alojado en Google Sheets

worksheet = gc.open(nameSheets).sheet1

rows = worksheet.get_all_values()

import pandas as pd
df = pd.DataFrame.from_records(rows)
# cambiar el nombre de las columans con la primera fila
new_header = df.iloc[0] 
df = df[1:]
df.columns = new_header 
df.rename(columns=df.iloc[0])
# reiniciar index
df = df.reset_index(drop=True)
df

,Marca temporal,Cedula,Copia Documento
0,29/03/2022 8:57:19,892374,https://drive.google.com/open?id=19ta7OQ4rLPjD...
1,29/03/2022 8:57:43,9912388,https://drive.google.com/open?id=1tJfIneYBIYOv...
2,29/03/2022 8:57:48,128383,
3,29/03/2022 8:57:55,7738217,
4,29/03/2022 8:58:14,48208,https://drive.google.com/open?id=1IgzQk-ScNDtT...


In [5]:
## Identificar el nombre de las columnas del Google Sheets

df.columns

Index(['Marca temporal', 'Cedula', 'Copia Documento'], dtype='object', name=0)

# Crear carpeta principal

Con el siguiente condigo, se crea la carpeta principal donde se van colcoar las copias de los documentos

Se debe incluir en el input el nombre de la carpeta principal que tendra la información

Nota: no iniciar el nombre de la carpeta con caracteres especiales (%"&$/='")

In [6]:
## Carpeta principal google Drive

folder_drive = '/content/drive/MyDrive/'

## Ejemplo de nombre de la carpeta 'Participantes'

New_folder = input()

A participantes


In [7]:
directory = folder_drive+'/'+New_folder

Con el siguiente codigo, se creara la carpeta principal. Este codigo tiene las sigueintes validaciones

*   Si la carpeta no existe y se colocaron los caracteres indicados en el paso anterior, se creara exitosamente
*   Si colocaron caracteres que no no corresponden, imprimira un mensaje con el error
*   Si la carpeta ya existe, imprimira un mensaje con ese resultado y la carpeta no se creara



In [8]:
if not os.path.exists(directory):
  try:
      os.mkdir(directory)
  except OSError:
      print("Error del directorio %s" % directory)
  else:
      print("Se ha creo el directorio: %s " % directory)
else:
  print("Ya existe la carpeta: %s " % directory)

Ya existe la carpeta: /content/drive/MyDrive//A participantes 


# Carpeta por participante



Con el siguiente Codigo, se crean automaticamente los nombres de los participantes, se suguiere usar el documento de identidad, pero puede aplicar con cualquiera

El nombre del participante, se debe seleccionar de las columnas existes en la base de datos, de lo contrario el script no funcionara

In [9]:
# Cuenta el numero de registros alojados en el Google Forms

contar = len(df.index)
contar 

5

In [10]:
# En el siguiente ejemplo, se selecciona la columna con las cedulas

# Documento de identidad

id = input()

Cedula


In [11]:
# crear carpeta por participante, con el documento de identidad

## Nota: Este codigo, si sobreescribe las carpetas si ya están creadas, tiene que tener cuidado

for i in range(contar):
  if not os.path.exists(directory+'/'+df[id][i]):
    try:
        os.mkdir(directory+'/'+df[id][i])
    except OSError:
        print("La creación del directorio %s falló o ya esta creado" % directory+'/'+df[id][i])
    else:
        print("Se ha creado el directorio: %s " % directory+'/'+df[id][i])

Se ha creado el directorio: /content/drive/MyDrive//A participantes /892374
Se ha creado el directorio: /content/drive/MyDrive//A participantes /9912388
Se ha creado el directorio: /content/drive/MyDrive//A participantes /128383
Se ha creado el directorio: /content/drive/MyDrive//A participantes /7738217
Se ha creado el directorio: /content/drive/MyDrive//A participantes /48208


# Copiar documentos en la carpeta del participante

Con el siguiente codigo, se realiza la copia del documento de en la carpeta designada

Se debe seleccionar el nombre de la columna en la base que contiene los documentos de identidad

In [12]:
# Ejemplo, se selecciona el nombre columna del archivo cargado

#columna = 'Cargue documento de identidad'

columna = input()

Copia Documento


In [13]:
# Esta libreria sirve para detectar el id de la carpeta creada en el anterior paso

!pip install kora
from kora.xattr import get_id

     |████████████████████████████████| 57 kB 3.4 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 


In [14]:
# Importa las librerias de Google Drive con el SDK para poder guardar información

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [15]:
# libreria de GOogle drive para actualizar archivos

from googleapiclient.http import MediaFileUpload

In [16]:
# Este servira como complemento para cambiar el nombre del archivo

titulo_documento = input()

Copia documento identidad


In [17]:
# Función que copia el documento a la carpeta del participante u organización creado

for i in range(contar):
  try:
    folder_id = get_id(directory+'/'+df[id][i])
    file_id = df[columna][i].split('=')[-1]
    file_metadata = {
      'name': str(df[id][i]+' - '+titulo_documento),
      'parents': str(folder_id),
    }
    file = drive_service.files().get(fileId=file_id,
                                  fields='parents').execute()
    previous_parents = ",".join(file.get('parents'))
    # Realiza copia del archivo
    file = drive_service.files().copy(fileId=file_id).execute()
    # Mueve el archivo copiado a la nueva carpeta
    file = drive_service.files().update(body=file_metadata,fileId=file_id, addParents=folder_id, removeParents=previous_parents, fields='id, parents').execute()
  except:
    None
